In [1]:
# Header block to include all modules that must be imported ahead of time
# Only needs to be run once per session, and each time a new module is added

import requests
from bs4 import BeautifulSoup
import json
import pandas as pd # this will need to be installed via command line first
import lxml # this will need to be installed via command line first(as well)
from babel.numbers import format_currency

In [36]:
# Method to get the salary cap hits for every player currently signed on an NFL team
# Creates a csv with columns player name, salary cap hit, and team location abbrviation 

def GetTeamSalaryCapHits(team):
    # Use requests to get the raw HTML response from overthecap.com
    OTCResponse = requests.get(f'https://overthecap.com/salary-cap/{team[0]}')
        
    # If we get a normal response proceed with data scraping
    if OTCResponse.status_code == 200:
        print(f"Request succeeded with status code {OTCResponse.status_code}")
        TeamNameCaptalized = (team[0].replace("-", " ")).title()
        print(f"Here are the Salary Cap hits for all players who are currently signed with the {TeamNameCaptalized}:")

        # Pandas read_html method allows the table contents to be put into a DataFrame
        TableTag = 'salary-cap-table contracted-players'
        DataFrames = pd.read_html(OTCResponse.content, attrs = {'class': TableTag})
        
        # Select the specific parts of the dataframe we want 
        SalariesDF = DataFrames[0][['Player', 'Cap Number']]
        if len(SalariesDF) >= 51:
            SalariesDF = SalariesDF.drop(51, axis=0)
        # Add team location abv to each player
        SalariesDF = SalariesDF.assign(TeamLOC = team[1].strip("0"))
        
        # Create list of salary cap hits, strip the non-numerical characters, add to dataframe
        CapIntsList = SalariesDF['Cap Number'].values
        CapIntsList = [i[0].strip('$,') for i in CapIntsList]
        SalariesDF['Cap Int'] = CapIntsList
        
        
        # Change the order of columns to make more sense semantically 
        new_order = ['Player', 'TeamLOC', 'Cap Number', 'Cap Int']
        SalariesDF = SalariesDF.loc[:, new_order]
        print(SalariesDF)
        
        # Specify filename and path
        csvPathName = f'./Salary_Lists/{TeamNameCaptalized}-Player-Salary-List.csv'
        
        # Create a .csv file with all the player names and their cap hits
        SalariesDF.to_csv(csvPathName, index = False, header = False)
        
        """
        # Read the csv back for testing purposes
        SalariesCSV = pd.read_csv(f'{TeamNameCaptalized}-Player-Salary-List.csv', header = None)
        SalariesCSV.columns = ['Player', 'TeamLOC', 'Cap Number', 'Cap Int']
        print(SalariesCSV)
        """
        

    # If we don't get a normal reponse, stop scraping
    else:
        print(f"Request failed with status code {OTCResponse.status_code}")
        return -1


In [37]:
# A list of links to Over The Cap Salary Cap pages for various teams
NFLTeams = [['buffalo-bills', 'BUF'], ['miami-dolphins', 'MIA'], ['new-england-patriots', 'NE0'], ['new-york-jets', 'NYJ'],
            ['baltimore-ravens', 'BAL'], ['cincinnati-bengals', 'CIN'], ['cleveland-browns', 'CLE'], ['pittsburgh-steelers', 'PIT'],
            ['houston-texans', 'HOU'], ['indianapolis-colts', 'IND'], ['jacksonville-jaguars', 'JAX'], ['tennessee-titans', 'TEN'], 
            ['denver-broncos', 'DEN'], ['kansas-city-chiefs', 'KC0'], ['las-vegas-raiders', 'LV0'], ['los-angeles-chargers', 'LAC'],
            ['dallas-cowboys', 'DAL'], ['new-york-giants', 'NYG'], ['philadelphia-eagles', 'PHI'], ['washington-commanders', 'WSH'],
            ['chicago-bears', 'CHI'], ['detroit-lions', 'DET'], ['green-bay-packers', 'GB0'], ['minnesota-vikings', 'MIN'],
            ['atlanta-falcons', 'ATL'], ['carolina-panthers', 'CAR'], ['new-orleans-saints', 'NO0'], ['tampa-bay-buccaneers', 'TB0'],
            ['arizona-cardinals', 'ARI'], ['los-angeles-rams', 'LAR'], ['san-francisco-49ers', 'SF0'], ['seattle-seahawks', 'SEA']
           ]

# Call the method defined above to get all the player data
def GetMultipleTeamSalaryCapHits(TeamList):
    for team in TeamList:
        GetTeamSalaryCapHits(team)

GetMultipleTeamSalaryCapHits(NFLTeams)
# GetTeamSalaryCapHits(['new-york-jets', 'NYJ'])

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Buffalo Bills:
                Player TeamLOC   Cap Number     Cap Int
                Player           Cap Number            
0           Josh Allen     BUF  $47,056,281  47,056,281
1         Stefon Diggs     BUF  $27,854,000  27,854,000
2           Von Miller     BUF  $23,799,000  23,799,000
3         Dion Dawkins     BUF  $16,609,833  16,609,833
4    Tre'Davious White     BUF  $16,443,740  16,443,740
5          Dawson Knox     BUF  $14,300,000  14,300,000
6          Matt Milano     BUF  $12,437,000  12,437,000
7        Taron Johnson     BUF  $12,409,833  12,409,833
8          Mitch Morse     BUF  $11,500,000  11,500,000
9        Rasul Douglas     BUF   $9,973,529   9,973,529
10           Ed Oliver     BUF   $9,725,000   9,725,000
11     Connor McGovern     BUF   $7,900,000   7,900,000
12        Jordan Poyer     BUF   $7,720,000   7,720,000
13        Deonte Harty     

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the New York Jets:
                      Player TeamLOC   Cap Number     Cap Int
                      Player           Cap Number            
0                C.J. Mosley     NYJ  $21,476,000  21,476,000
1           Quinnen Williams     NYJ  $20,400,000  20,400,000
2            Laken Tomlinson     NYJ  $18,880,000  18,880,000
3              Aaron Rodgers     NYJ  $17,161,112  17,161,112
4        John Franklin-Myers     NYJ  $16,364,000  16,364,000
5                  D.J. Reed     NYJ  $15,636,176  15,636,176
6               Allen Lazard     NYJ  $12,184,000  12,184,000
7                C.J. Uzomah     NYJ  $11,221,668  11,221,668
8                Zach Wilson     NYJ  $11,184,307  11,184,307
9              Tyler Conklin     NYJ   $9,338,334   9,338,334
10             Ahmad Gardner     NYJ   $9,122,945   9,122,945
11           Quincy Williams     NYJ   $7,150,000   7,150,000

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Cleveland Browns:
                      Player TeamLOC   Cap Number     Cap Int
                      Player           Cap Number            
0             Deshaun Watson     CLE  $63,977,000  63,977,000
1               Amari Cooper     CLE  $23,776,000  23,776,000
2                Denzel Ward     CLE  $23,498,118  23,498,118
3              Myles Garrett     CLE  $20,168,895  20,168,895
4                 Nick Chubb     CLE  $15,825,000  15,825,000
5              Jedrick Wills     CLE  $14,581,352  14,581,352
6               Joel Bitonio     CLE  $12,135,000  12,135,000
7               Jack Conklin     CLE  $12,025,824  12,025,824
8               Wyatt Teller     CLE  $11,610,300  11,610,300
9                David Njoku     CLE   $9,645,000   9,645,000
10          Dalvin Tomlinson     CLE   $6,502,000   6,502,000
11           Greg Newsome II     CLE   $4,056,422   4,056,

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Jacksonville Jaguars:
                  Player TeamLOC   Cap Number     Cap Int
                  Player           Cap Number            
0         Christian Kirk     JAX  $24,236,941  24,236,941
1        Brandon Scherff     JAX  $23,967,000  23,967,000
2       Foyesade Oluokun     JAX  $21,750,668  21,750,668
3           Cam Robinson     JAX  $21,168,301  21,168,301
4     Folorunso Fatukasi     JAX  $12,787,922  12,787,922
5       Rayshawn Jenkins     JAX  $12,564,000  12,564,000
6        Trevor Lawrence     JAX  $11,707,018  11,707,018
7       Darious Williams     JAX  $11,500,000  11,500,000
8              Zay Jones     JAX  $10,752,628  10,752,628
9          Travon Walker     JAX  $10,192,534  10,192,534
10           Evan Engram     JAX   $7,000,000   7,000,000
11        Davon Hamilton     JAX   $5,655,294   5,655,294
12  Roy Robertson-Harris     JAX   $4,800,000   

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Kansas City Chiefs:
                      Player TeamLOC   Cap Number     Cap Int
                      Player           Cap Number            
0            Patrick Mahomes      KC  $57,358,269  57,358,269
1                 Joe Thuney      KC  $26,971,713  26,971,713
2              Jawaan Taylor      KC  $24,725,000  24,725,000
3               Travis Kelce      KC  $15,463,709  15,463,709
4                Justin Reid      KC  $14,250,000  14,250,000
5   Marquez Valdes-Scantling      KC  $14,000,000  14,000,000
6            Charles Omenihu      KC  $10,970,000  10,970,000
7            Harrison Butker      KC   $4,568,797   4,568,797
8             Trent McDuffie      KC   $3,816,132   3,816,132
9                Chris Jones      KC   $3,400,080   3,400,080
10          George Karlaftis      KC   $3,255,130   3,255,130
11      Felix Anudike-Uzomah      KC   $2,685,868   2,68

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Dallas Cowboys:
                   Player TeamLOC   Cap Number     Cap Int
                   Player           Cap Number            
0            Dak Prescott     DAL  $59,455,000  59,455,000
1             Zack Martin     DAL  $28,500,000  28,500,000
2       Demarcus Lawrence     DAL  $20,445,000  20,445,000
3             CeeDee Lamb     DAL  $17,991,000  17,991,000
4            Trevon Diggs     DAL  $15,367,647  15,367,647
5          Michael Gallup     DAL  $13,850,000  13,850,000
6          Terence Steele     DAL  $11,000,000  11,000,000
7           Brandin Cooks     DAL  $10,000,000  10,000,000
8          Donovan Wilson     DAL   $7,370,588   7,370,588
9             Tyron Smith     DAL   $6,005,000   6,005,000
10          Micah Parsons     DAL   $5,434,479   5,434,479
11             Trey Lance     DAL   $5,310,717   5,310,717
12   Leighton Vander Esch     DAL   $4,3

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Chicago Bears:
                 Player TeamLOC   Cap Number     Cap Int
                 Player           Cap Number            
0          Montez Sweat     CHI  $25,085,294  25,085,294
1      Tremaine Edmunds     CHI  $22,437,500  22,437,500
2         Eddie Jackson     CHI  $18,140,000  18,140,000
3            D.J. Moore     CHI  $16,050,000  16,050,000
4        Cody Whitehair     CHI  $13,250,000  13,250,000
5             Cole Kmet     CHI  $11,600,000  11,600,000
6            Nate Davis     CHI  $11,352,941  11,352,941
7       DeMarcus Walker     CHI   $8,666,666   8,666,666
8          T.J. Edwards     CHI   $7,500,000   7,500,000
9         Justin Fields     CHI   $6,004,712   6,004,712
10       Darnell Wright     CHI   $4,765,516   4,765,516
11         Cairo Santos     CHI   $4,260,000   4,260,000
12      Andrew Billings     CHI   $3,500,000   3,500,000
13        Te

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Atlanta Falcons:
                   Player TeamLOC   Cap Number     Cap Int
                   Player           Cap Number            
0           Jake Matthews     ATL  $21,773,713  21,773,713
1           Grady Jarrett     ATL  $20,375,000  20,375,000
2         Chris Lindstrom     ATL  $18,000,000  18,000,000
3        Jessie Bates III     ATL  $17,500,000  17,500,000
4          David Onyemata     ATL  $12,500,000  12,500,000
5            Kaleb McGary     ATL  $12,500,000  12,500,000
6            A.J. Terrell     ATL  $12,344,000  12,344,000
7              Kyle Pitts     ATL  $10,471,522  10,471,522
8             Jonnu Smith     ATL   $9,000,000   9,000,000
9         Taylor Heinicke     ATL   $8,960,000   8,960,000
10           Kaden Elliss     ATL   $8,500,000   8,500,000
11           Drake London     ATL   $5,873,038   5,873,038
12           Younghoe Koo     ATL   $5,

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Tampa Bay Buccaneers:
                   Player TeamLOC   Cap Number     Cap Int
                   Player           Cap Number            
0            Chris Godwin      TB  $27,534,000  27,534,000
1                Vita Vea      TB  $20,965,063  20,965,063
2           Carlton Davis      TB  $20,370,275  20,370,275
3           Tristan Wirfs      TB  $18,606,582  18,606,582
4              Jamel Dean      TB  $14,666,353  14,666,353
5         Shaquil Barrett      TB  $14,190,882  14,190,882
6            Russell Gage      TB  $13,350,668  13,350,668
7              Mike Evans      TB  $12,198,000  12,198,000
8             Ryan Jensen      TB   $5,977,000   5,977,000
9          Anthony Nelson      TB   $5,384,000   5,384,000
10     Joe Tryon-Shoyinka      TB   $3,554,487   3,554,487
11         Calijah Kancey      TB   $3,293,971   3,293,971
12          Lavonte David      TB 

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the San Francisco 49Ers:
                 Player TeamLOC   Cap Number     Cap Int
                 Player           Cap Number            
0        Trent Williams      SF  $31,568,542  31,568,542
1          Deebo Samuel      SF  $28,633,765  28,633,765
2         Arik Armstead      SF  $28,353,236  28,353,236
3           Fred Warner      SF  $24,469,000  24,469,000
4         George Kittle      SF  $21,956,575  21,956,575
5       Charvarius Ward      SF  $18,401,000  18,401,000
6        Javon Hargrave      SF  $15,430,882  15,430,882
7             Nick Bosa      SF  $14,671,000  14,671,000
8   Christian McCaffrey      SF  $14,144,000  14,144,000
9         Brandon Aiyuk      SF  $14,124,000  14,124,000
10         Dre Greenlaw      SF   $9,602,970   9,602,970
11        Kyle Juszczyk      SF   $7,588,750   7,588,750
12         Jake Brendel      SF   $4,692,000   4,692,000
13    